AIM: processing of the CPOM dataset (Armitage extended) data and the ssh model data to save comparable time series in the BGR box. 

In [1]:
# standart librairies
import os,sys
import numpy as np
import xarray as xr
import pandas as pd
import datetime

# plot
import cartopy
import cartopy.feature as cfeature
import cartopy.crs as ccrs
import matplotlib.path as mpath
import matplotlib.pyplot as plt
from matplotlib.colors import Colormap
import matplotlib.colors as mcolors
import matplotlib.dates as mdates
import matplotlib.cm as cm
import matplotlib.dates as mdates
import matplotlib.ticker as mticker
from matplotlib import ticker

from matplotlib.colors import from_levels_and_colors
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

from scipy.interpolate import griddata


In [2]:
from dask.distributed import Client
c = Client()
c

/linkhome/rech/genlop01/uor98hu/.local/lib/python3.9/site-packages/distributed/node.py:183: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 41575 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:41575/status,
Dashboard: http://127.0.0.1:41575/status,Workers: 5
Total threads: 10,Total memory: 20.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:34357,Workers: 5
Dashboard: http://127.0.0.1:41575/status,Total threads: 10
Started: Just now,Total memory: 20.00 GiB
Comm: tcp://127.0.0.1:40231,Total threads: 2
Dashboard: http://127.0.0.1:46197/status,Memory: 4.00 GiB
Nanny: tcp://127.0.0.1:45361,


# loading CPOM arctic data

In [3]:
chunk_size = {'lat':25,'lon':80}
diri = '/gpfswork/rech/cli/uor98hu/DATA4VALIDATION/Armitage/'
namo = 'Full_DOT_data_Arco.nc'
CPOMdat = xr.open_dataset(diri+namo,chunks=chunk_size)
# 2011-01 to 2020-12: change time unit
time = pd.date_range(start='01/2011', end='01/2021', freq='M')
CPOMdata = CPOMdat.rename({'time':'time_counter'})
CPOMnav_lat = CPOMdata.lats
CPOMnav_lon = CPOMdata.lons
CPOMdata = CPOMdata.assign_coords(nav_lon=CPOMnav_lon, nav_lat=CPOMnav_lat, time_counter=time)
sla = CPOMdata.Sea_level_anomaly


## interpolating the BGRmask of the model onto the CPOM product grid

### test : interpolating on the whole grid

In [5]:
#### model 
BGRmask = xr.open_dataset('/gpfswork/rech/cli/uor98hu/MYDATA/TRANSPORT_WP1/BGRmaskobs.nc').BGRmask.squeeze()

dirmask="/gpfswork/rech/cli/rcli002/eORCA025.L75/eORCA025.L75-I/"
mesh_hgr = xr.open_dataset(dirmask+'mesh_hgr.nc').squeeze().rename({"nav_lev":"deptht"}).reset_coords(names='time_counter', drop=True)

nav_lon =  mesh_hgr.nav_lon.fillna(0)
nav_lat =  mesh_hgr.nav_lat.fillna(0)

model_grid = (nav_lat.values.ravel(), nav_lon.values.ravel() )

BGRmask = BGRmask.assign_coords(nav_lon=nav_lon, nav_lat=nav_lat)

In [20]:
### apply interpolation
# scipy.interpolate.griddata
grid_z0 = griddata(model_grid, BGRmask.values.ravel(), (CPOMnav_lat.values,CPOMnav_lon.values), method='nearest')
grid_z0[252,108] = 0
grid_z0[251,109] = 0
grid_z0[252,109] = 0
grid_z0[253,109] = 0
grid_z0[252,110] = 0
grid_z0[253,110] = 0
CPOMmaskcorr = xr.DataArray(
    data=grid_z0,
    dims=["x", "y"],
    coords=dict(
        nav_lon=(["x", "y"], CPOMnav_lon.data),
        nav_lat=(["x", "y"], CPOMnav_lat.data)))
dsCPOMmaskcorr = CPOMmaskcorr.to_dataset(name='BGRmaskCPOM')

In [17]:
### apply interpolation
# scipy.interpolate.griddata
grid_z0 = griddata(model_grid, BGRmask.values.ravel(), (CPOMnav_lat.values,CPOMnav_lon.values), method='nearest')

CPOMmask = xr.DataArray(
    data=grid_z0,
    dims=["x", "y"],
    coords=dict(
        nav_lon=(["x", "y"], CPOMnav_lon.data),
        nav_lat=(["x", "y"], CPOMnav_lat.data)))
dsCPOMmask = CPOMmask.to_dataset(name='BGRmaskCPOM')

In [21]:
# dsCPOMmask.to_netcdf("/gpfswork/rech/cli/uor98hu/DATA4VALIDATION/Armitage/Full_DOT_data_Arco_BGRmask.nc", mode='w') 
dsCPOMmaskcorr.to_netcdf("/gpfswork/rech/cli/uor98hu/DATA4VALIDATION/Armitage/Full_DOT_data_Arco_BGRmaskcorr.nc", mode='w') 